# Data Engineer Challenge

In [3]:
import pandas as pd
import json

from pandas import json_normalize

# Preparación de los datos

Debido a que el archivo de origen contenía registros malformados, fue necesario procesar el archivo línea por línea, en caso de que el archivo hubiese venido sin malformados se podría haber leido el archivo utilizando *pd.read_json*

In [12]:
tweets_path = r'../src/resources/farmers-protest-tweets-2021-2-4.json'

In [ ]:
tweets, bad_lines = [], []
with open(tweets_path, 'r') as file:
    for line in file:
        try:
          tweets.append(json.loads(line))
        except:
          bad_lines.append(line)

In [11]:
tweets_flat_df = json_normalize(tweets)

# Desafío

## Los top 10 tweets más retweeted.

## Los top 10 users en función a la cantidad de tweets que emitieron.

## Los top 10 días donde hay más tweets.

   ## Los top 10 hashtags más usados.

## Los top 10 emojis más usados.

## Los top 10 users más influyentes en función de lo retweeted de sus tweets.